In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from time import sleep

%matplotlib inline

In [41]:
#Import the dataset
data_train=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/CSV/2 patterns (P1-P5)/LẦN 1/Train_56.000.csv')
data_test=pd.read_csv('/content/drive/MyDrive/N12-Kĩ thuật giấu tin/CSV/2 patterns (P1-P5)/LẦN 1/Test_14.000.csv')

X_train=data_train.iloc[:56000,[1,2,3,4,5]].values
Y_train=data_train.iloc[:56000,0].values

X_test=data_test.iloc[:14000,[1,2,3,4,5]].values
Y_test=data_test.iloc[:14000,0].values


In [42]:
X_train, Y_train, X_test, Y_test = list(map(lambda x: np.array(x, dtype=np.float64), [X_train, Y_train, X_test, Y_test]))

In [43]:
data_train.head()

,LABEL,ip.proto,ip.id,tcp.seq_raw,frame.len,frame.cap_len
0,0,6,47010,698643808,1420,66
1,1,6,24320,1862270976,56,56
2,1,6,52834,1159528989,166,166
3,1,6,45038,1854525181,126,126
4,0,6,4160,1177938840,54,54


In [44]:
X_train[0]

array([6.00000000e+00, 4.70100000e+04, 6.98643808e+08, 1.42000000e+03,
       6.60000000e+01])

In [45]:
max_words = 65527
max_len = 5
# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
#sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)
sequences_matrix = X_train
Y_train = Y_train.reshape(-1, 1)

In [46]:
tcp_train = X_train[:, 2]
tcp_test = X_test[:, 2]

In [47]:
#Trộn train+test
tcp = np.concatenate([tcp_train, tcp_test])

In [48]:
tcp.shape

(70000,)

In [49]:
#Sort tcp_test
tcp_set = list(set(tcp))
tcp_sort = list(np.sort(tcp_set))
len(tcp_set)

52832

In [50]:
#Đánh index value tcp_train và tcp_test= index trong tcp+sort
result_train = []
result_test = []
for value in tcp_train:
  index = tcp_sort.index(value)
  result_train.append(index)

for value in tcp_test:
  index = tcp_sort.index(value)
  result_test.append(index)

result_train = np.array(result_train)
result_test = np.array(result_test)

In [51]:
np.max(result_test)

52830

In [52]:
X_train[:, 2] = result_train
X_test[:, 2] = result_test

In [53]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    # layer = Merge([max_len, context_model], mode="dot", dot_axes=0)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [54]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 5)]               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 5, 50)             3276350   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [57]:
start = time.time()
model.fit(sequences_matrix,Y_train,batch_size=1024,epochs=10,
          validation_split=0.2)
end = time.time()

Epoch 1/10
44/44 [==============================] - 3s 24ms/step - loss: 0.2531 - accuracy: 0.9376 - val_loss: 0.0109 - val_accuracy: 0.9987
Epoch 2/10
44/44 [==============================] - 1s 17ms/step - loss: 0.0041 - accuracy: 0.9993 - val_loss: 0.0045 - val_accuracy: 0.9991
Epoch 3/10
44/44 [==============================] - 1s 15ms/step - loss: 0.0017 - accuracy: 0.9997 - val_loss: 0.0037 - val_accuracy: 0.9992
Epoch 4/10
44/44 [==============================] - 1s 15ms/step - loss: 9.9605e-04 - accuracy: 0.9998 - val_loss: 0.0038 - val_accuracy: 0.9993
Epoch 5/10
44/44 [==============================] - 1s 16ms/step - loss: 5.0802e-04 - accuracy: 0.9998 - val_loss: 0.0042 - val_accuracy: 0.9993
Epoch 6/10
44/44 [==============================] - 1s 15ms/step - loss: 2.5850e-04 - accuracy: 0.9999 - val_loss: 0.0048 - val_accuracy: 0.9993
Epoch 7/10
44/44 [==============================] - 1s 15ms/step - loss: 5.3221e-05 - accuracy: 1.0000 - val_loss: 0.0048 - val_accuracy: 0.99

In [59]:
exe_time = end - start
print(exe_time)

8.837362051010132


In [63]:
start = time.time()
model.evaluate(X_test, Y_test)
end = time.time()
exe_time = end - start
print(exe_time)

438/438 [==============================] - 1s 3ms/step - loss: 0.0073 - accuracy: 0.9973
1.1854941844940186


In [64]:
model.predict(np.array([X_test[4]]))

array([[1.]], dtype=float32)

In [65]:
Y_test[4]

1.0